# Introduction to Machine Learning Pipeline: 
# An Example Using House Prediction Dataset¶

## Goal

This notebook is designed to **develop pipelines from data preparation to model prediction** that help Real Estate Sales Agents accurately predict house prices (i.e., SalePrice). The objectives are:

1. To enable agents to better assess the value of a house when owners are looking to sell.
2. To help agents understand the budget of buyers based on their descriptions of an ideal house.

A detailed explanation of each step can be found in my previous notebook on [Kaggle](https://www.kaggle.com/code/somebodynamedalexis/introduction-to-ml-prediction) or [Github](https://nbviewer.org/github/Zihac/Introduction-to-ML-Models/blob/main/ML_Prediction/house-price-prediction-begining.ipynb). In this notebook, I will focus on explaining the code related to pipeline building.

In [1]:
import numpy as np 
import pandas as pd

# code if you are running on local PC
import opendatasets as od
# od.download("https://www.kaggle.com/competitions/house-prices-advanced-regression-techniques/data")
train = pd.read_csv('../Kaggle/house-prices-advanced-regression-techniques/train.csv')
test = pd.read_csv('../Kaggle/house-prices-advanced-regression-techniques/test.csv')

In [ ]:
# code if you are running it on Kaggle

# import numpy as np # linear algebra
# import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# import os
# for dirname, _, filenames in os.walk('/kaggle/input'):
#     for filename in filenames:
#         print(os.path.join(dirname, filename))

# train = pd.read_csv('../input/house-prices-advanced-regression-techniques/train.csv')
# test = pd.read_csv('../input/house-prices-advanced-regression-techniques/test.csv')

In [2]:
pd.set_option('display.max_columns', 500)

train.head(1)

,Id,MSSubClass,MSZoning,LotFrontage,LotArea,Street,Alley,LotShape,LandContour,Utilities,LotConfig,LandSlope,Neighborhood,Condition1,Condition2,BldgType,HouseStyle,OverallQual,OverallCond,YearBuilt,YearRemodAdd,RoofStyle,RoofMatl,Exterior1st,Exterior2nd,MasVnrType,MasVnrArea,ExterQual,ExterCond,Foundation,BsmtQual,BsmtCond,BsmtExposure,BsmtFinType1,BsmtFinSF1,BsmtFinType2,BsmtFinSF2,BsmtUnfSF,TotalBsmtSF,Heating,HeatingQC,CentralAir,Electrical,1stFlrSF,2ndFlrSF,LowQualFinSF,GrLivArea,BsmtFullBath,BsmtHalfBath,FullBath,HalfBath,BedroomAbvGr,KitchenAbvGr,KitchenQual,TotRmsAbvGrd,Functional,Fireplaces,FireplaceQu,GarageType,GarageYrBlt,GarageFinish,GarageCars,GarageArea,GarageQual,GarageCond,PavedDrive,WoodDeckSF,OpenPorchSF,EnclosedPorch,3SsnPorch,ScreenPorch,PoolArea,PoolQC,Fence,MiscFeature,MiscVal,MoSold,YrSold,SaleType,SaleCondition,SalePrice
0,1,60,RL,65.0,8450,Pave,NaN,Reg,Lvl,AllPub,Inside,Gtl,CollgCr,Norm,Norm,1Fam,2Story,7,5,2003,2003,Gable,CompShg,VinylSd,VinylSd,BrkFace,196.0,Gd,TA,PConc,Gd,TA,No,GLQ,706,Unf,0,150,856,GasA,Ex,Y,SBrkr,856,854,0,1710,1,0,2,1,3,1,Gd,8,Typ,0,NaN,Attchd,2003.0,RFn,2,548,TA,TA,Y,0,61,0,0,0,0,NaN,NaN,NaN,0,2,2008,WD,Normal,208500


# Pipeline for Data Preprocessing

In [3]:
# Split the data into training and testing
y_train = train['SalePrice']
X_train = train.drop(['SalePrice'], axis=1)

# get feature names
time = train[['MoSold','YrSold']].columns.to_list()
num_attribs = train.select_dtypes(include=np.number).drop(['MoSold','YrSold','SalePrice'], axis=1).columns.to_list()
cat_attribs = train.select_dtypes(include="object").columns.to_list()

In [4]:
time

['MoSold', 'YrSold']

For **numeric features** (num_pipeline): we fill the missing values with the median (imputer), then standardize the numeric features (std_scaler).

For **categorical features** (cate_pipeline): we fill the missing values with the mode (imputer), then use a one-hot encoder to transform all categorical features into binary columns (cat).


In [5]:
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import StandardScaler, OneHotEncoder

num_pipeline = Pipeline([
    ("imputer", SimpleImputer(strategy="median")),
    ("std_scaler", StandardScaler()),
])


cate_pipeline = Pipeline([
    ("imputer", SimpleImputer(strategy="most_frequent")), #mode
    ("cat", OneHotEncoder()),
])

# Pipeline for Feature Engineering 

## Time features

- **During fit (only training dataset)**:
    - If time_series=True, the method:
        1. Gets the median of house prices by year-month.
        2. Calculates the year-month seasonal and financial crisis indicators.
        3. Stores the resulting dataframe in self.temp.

- **During transform (training + testing)**:
    - If self.time_series and self.temp is not None:
        1. Retrieves the dataframe stored in self.temp.
        2. Merges the result back to the input data X.
        3. Returns the financial crisis indicator and seasonal components.
    - Else, returns the original input data X without calculating the time features.

In [6]:
from sklearn.base import BaseEstimator,TransformerMixin
from statsmodels.tsa.seasonal import seasonal_decompose

class TimeSeries(BaseEstimator, TransformerMixin):
    def __init__(self, time_series=True):
        # time_series=Ture-->run if; by default is True
        # time_series=False-->run else
        self.time_series = time_series
        self.temp = None

    def fit(self, X, y=None):
        if self.time_series and y is not None:
            X = X.copy()
            X['SalePrice'] = y
            X['YearMonthSold'] = pd.to_datetime(X['YrSold'].astype(str) + '-' + X['MoSold'].astype(str))

            # Median SalePrice for each year-month
            temp = X.groupby('YearMonthSold')['SalePrice'].median().reset_index()
            temp.set_index('YearMonthSold', inplace=True)

            # Decompose the time series to extract seasonal and trend components
            decompose = seasonal_decompose(temp['SalePrice'], model='multiplicative', period=12)
            temp['seasonal'] = decompose.seasonal
            temp['trend'] = decompose.trend

            # Create indicator for financial crisis
            temp.reset_index(inplace=True)
            temp['YearMonthSold'] = temp['YearMonthSold'].dt.strftime('%Y-%m')
            temp['is_crisis'] = 0
            temp.loc[temp['YearMonthSold'] >= '2008-01', 'is_crisis'] = 1
            temp['YearMonthSold'] = pd.to_datetime(temp['YearMonthSold'], format='%Y-%m')
            self.temp = temp

        return self
    
    def transform(self, X):
        if self.time_series and self.temp is not None:
            X = X.copy()
            X['YearMonthSold'] = pd.to_datetime(X['YrSold'].astype(str) + '-' + X['MoSold'].astype(str))
            X = X.merge(self.temp[['YearMonthSold', 'is_crisis', 'seasonal']], on='YearMonthSold', how='left')
            return X[['is_crisis', 'seasonal']]
        else:
            return X

*The ColumnTransformer ensures that:*

If the testing features' column order is different from the training input feature columns, the output sequence will follow the order specified in the ColumnTransformer, ensuring consistent preprocessing.

In [7]:
data_preprocess = ColumnTransformer([
   ("time", TimeSeries(),time),
   ("num", num_pipeline, num_attribs), # num_pipeline returns a dense matrix
   ("cat", cate_pipeline, cat_attribs), # OneHotEncoder returns a sparse matrix
])

# Pipeline for Feature Selection

In this session, I jointly select numerical and categorical features using **mutual_info_regression**. My previous notebook shows that the features selected by mutual_info_regression are similar to those selected based on correlation. 

## SelectKBest 
- Select the top 10 features based on mutual_info_regression.
    - To simplify the selection process, we directly use SelectKBest(score_func=mutual_info_regression, k=10). However, to return the top 10 features with their column names, we need to customize it.
 
    
- **During fit (only training dataset)**:
    1. Fit the SelectKBest based on the criterion mutual_info_regression, using the training data with features (X) and target (y).
    2. Get the indices of the selected features (mask).
    3. Store the column names (self.selected_features_) and SelectKBest object (self.selector) after training.

- **During transform (training + testing)**:
    1. Apply the same feature selection on the data (X).
    2. Return a DataFrame with the selected features and their corresponding column names.

In [8]:
from sklearn.feature_selection import SelectKBest, mutual_info_regression

In [9]:
class SelectK(BaseEstimator, TransformerMixin):
    def __init__(self, score_func, k):
        self.k = k
        self.score_func = score_func
        self.selector = SelectKBest(score_func=self.score_func, k=self.k)
        self.selected_features_ = None

    def fit(self, X, y=None):
        self.selector.fit(X, y)
        mask = self.selector.get_support()  # Get boolean mask of selected features
        self.selected_features_ = X.columns[mask]
        return self

    def transform(self, X):
        X_selected = self.selector.transform(X)
        return pd.DataFrame(X_selected, columns=self.selected_features_)

## Remove Highly Correlated Features

- Remove features that show high Pearson correlations.

- **During fit (only training dataset)**:
    1. Get the correlation matrix with absolute values (corr_matrix).
    2. Consider only the upper triangle of the matrix (upper).
    3. If the value is greater than the threshold, save the corresponding column in the dropping list (self.to_drop).
    
- **During transform (training + testing)**:
    1. If the drop list is non-null, drop the selected columns from the original data X.


In [23]:
#  train.corr().abs()
# np.ones(train.shape)

In [10]:
# Custom transformer to drop highly correlated features
class DropHighlyCorrelatedFeatures(BaseEstimator, TransformerMixin):
    def __init__(self, threshold):
        self.threshold = threshold
        self.to_drop = None

    def fit(self, X, y=None):
        # Create a DataFrame to calculate correlations
        df = pd.DataFrame(X)
        corr_matrix = df.corr().abs()
        # Select the upper triangle of the correlation matrix
            # np.triu:  sets all elements below the k-th diagonal to zero
                # k=1:  exclude the diagonal itself and only consider elements above it
            # np.ones: creates a matrix of ones to identify the positions in the correlation matrix
        upper = corr_matrix.where(np.triu(np.ones(corr_matrix.shape), k=1).astype(bool))
        
        # Find features with correlation greater than the threshold
        self.to_drop = [column for column in upper.columns if any(upper[column] > self.threshold)]
        return self

    def transform(self, X):
        if self.to_drop is not None:
            X = pd.DataFrame(X).drop(columns=self.to_drop)
        return X

Create pipeline for feature selections. 

In [11]:
feature_selection = Pipeline([
    ('selectK', SelectK(score_func=mutual_info_regression, k=10)),
    ('removeHigh', DropHighlyCorrelatedFeatures(threshold=0.8))
])

# Pipeline for Feature Names

After applying the pipelines in *data_preprocess*, the return will be in the form of a sparse matrix. To customize a pipeline to return the result in DataFrame format with column names, I built the DataFrameConverter class:


- **During fit (only training dataset)**:
    - Do nothing, return the object stored in self.
- **During transform (training + testing)**:
    - if self.time_series = True:
        - Get the column names of time, numeric, and categorical features.
        - Return the data X in DataFrame format with column names.
    - if  self.time_series = False:
        - Get the column names of numeric and categorical features.
        - Return the data X in DataFrame format with column names.

In [12]:
check = data_preprocess.fit_transform(X_train, y_train)
check

<1460x289 sparse matrix of type '<class 'numpy.float64'>'
	with 116157 stored elements in Compressed Sparse Row format>

In [13]:
time_attribs = ['is_crisis', 'seasonal']
time_attribs

['is_crisis', 'seasonal']

In [14]:
class DataFrameConverter(BaseEstimator, TransformerMixin):
    def __init__(self, time_attribs, num_attribs, cat_attribs, time_series=True):
        self.time_series=time_series
        self.num_attribs = num_attribs
        self.cat_attribs = cat_attribs
        self.time_attribs = time_attribs
    
    def fit(self, X, y=None):
        return self
    
    def transform(self, X):
        if self.time_series:
            # Get the feature names from the transformers
            time_feature_names = self.time_attribs
            num_feature_names = self.num_attribs
            cat_feature_names = data_preprocess.named_transformers_['cat']['cat'].get_feature_names_out(self.cat_attribs).tolist()

            # Combine all feature names
            all_feature_names = time_feature_names + num_feature_names + cat_feature_names
            # Convert the sparse matrix to a dense array (if necessary)
            X_dense = X.toarray() if hasattr(X, 'toarray') else X
            # Create DataFrame with the transformed data and feature names
            return pd.DataFrame(X_dense, columns=all_feature_names)
        else:
            all_feature_names = num_feature_names + cat_feature_names
            X_dense = X.toarray() if hasattr(X, 'toarray') else X
            return pd.DataFrame(X_dense, columns=all_feature_names)


# Combine Pipelines before Model Selection

In [15]:
full_pipeline = Pipeline([
    ('data_preprocess', data_preprocess),
    ('dataframe_converter', DataFrameConverter(time_attribs, num_attribs, cat_attribs)),
    ('feature_selection', feature_selection),
])

In [16]:
X_train_prepared = full_pipeline.fit_transform(X_train, y_train)
X_train_prepared

,MSSubClass,OverallQual,YearBuilt,TotalBsmtSF,GrLivArea,FullBath,GarageYrBlt,GarageCars
0,0.073375,0.651479,1.050994,-0.459303,0.370333,0.789741,1.017598,0.311725
1,-0.872563,-0.071836,0.156734,0.466465,-0.482512,0.789741,-0.107927,0.311725
2,0.073375,0.651479,0.984752,-0.313369,0.515013,0.789741,0.934226,0.311725
3,0.309859,0.651479,-1.863632,-0.687324,0.383659,-1.026041,0.809167,1.650307
4,0.073375,1.374795,0.951632,0.199680,1.299326,0.789741,0.892540,1.650307
...,...,...,...,...,...,...,...,...
1455,0.073375,-0.071836,0.918511,-0.238122,0.250402,0.789741,0.850854,0.311725
1456,-0.872563,-0.071836,0.222975,1.104925,1.061367,0.789741,-0.024555,0.311725
1457,0.309859,0.651479,-1.002492,0.215641,1.569647,0.789741,-1.566941,-1.026858
1458,-0.872563,-0.795151,-0.704406,0.046905,-0.832788,-1.026041,-1.191766,-1.026858


# Pipelines for Model Selection

- In this section, I build a class *ModelSelector* to find the best combination of base models for the stacking model.

- **During fit (only training dataset)**:
    1. For models considered as base models, run cross-validation on the training set for each model using `cross_val_score` and evaluate performance based on mean squared error (`mean_score`).
    2. Sort the models by mean squared error and remove the model with the worst performance.
    3. Use the selected models as base models for the stacking model, set linear regression as the meta model, and train based on cross-validation (`StackingRegressor(estimators=estimators, final_estimator=level1, cv=5)`).
    4. Fit the training data in the stacking model.

- **During predict (only testing dataset)**:
    - Return the predicted house prices based on the testing data.

In [17]:
from sklearn.model_selection import cross_val_score, RepeatedKFold, GridSearchCV
from sklearn.neighbors import KNeighborsRegressor
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor, StackingRegressor
from sklearn.svm import SVR
from sklearn.linear_model import LinearRegression

In [18]:
# Custom transformer to evaluate models and drop the worst performer
class ModelSelector(BaseEstimator, TransformerMixin):
    def __init__(self, models):
        self.models = models
        self.best_models = None

    def fit(self, X, y=None):
        model_scores = []
        for name, model in self.models:
            scores = cross_val_score(model, X, y, scoring='neg_mean_squared_error', cv=RepeatedKFold(n_splits=5, n_repeats=3, random_state=123), n_jobs=-1)
            mean_score = np.sqrt(np.abs(scores)).mean()
            model_scores.append((name, model, mean_score))

        # Sort models by their mean score (ascending)
        model_scores.sort(key=lambda x: x[2])
        # Drop the worst performing model
        self.best_models = model_scores[:-1]

        # Create the stacking model
        estimators = [(name, model) for name, model, score in self.best_models]
        level1 = LinearRegression()
        self.stacking_model = StackingRegressor(estimators=estimators, final_estimator=level1, cv=5)

        # Fit the stacking model
        self.stacking_model.fit(X, y)
        return self

    def predict(self, X):
        # Use predict to get the predictions from the stacking model
        return self.stacking_model.predict(X)

    def get_stacking(self):
        return self.stacking_model

    def get_best_models(self):
        return self.best_models

In [19]:
# Define the models
models = [
    ('dt', DecisionTreeRegressor()),
    ('rf', RandomForestRegressor()),
    ('knn', KNeighborsRegressor()),
    ('svr', SVR())
]

In [21]:
# Initialize and fit the ModelSelector
model_selector = ModelSelector(models)
model_selector.fit(X_train_prepared, y_train)

ModelSelector(models=[('dt', DecisionTreeRegressor()),
                      ('rf', RandomForestRegressor()),
                      ('knn', KNeighborsRegressor()), ('svr', SVR())])

In [22]:
best_models = model_selector.get_best_models()
best_models

[('rf', RandomForestRegressor(), 31904.47616957313),
 ('knn', KNeighborsRegressor(), 34785.59352123518),
 ('dt', DecisionTreeRegressor(), 41167.24014524628)]

In [23]:
model_selector.get_stacking()

StackingRegressor(cv=5,
                  estimators=[('rf', RandomForestRegressor()),
                              ('knn', KNeighborsRegressor()),
                              ('dt', DecisionTreeRegressor())],
                  final_estimator=LinearRegression())

# Make Prediction

In [24]:
X_test_prepared = full_pipeline.transform(test)

In [25]:
X_test_prepared

,MSSubClass,OverallQual,YearBuilt,TotalBsmtSF,GrLivArea,FullBath,GarageYrBlt,GarageCars
0,-0.872563,-0.795151,-0.340077,-0.400017,-1.179256,-1.026041,-0.733219,-1.026858
1,-0.872563,-0.071836,-0.439440,0.619239,-0.354966,-1.026041,-0.858277,-1.026858
2,0.073375,-0.795151,0.852269,-0.295127,0.216136,0.789741,0.767481,0.311725
3,0.073375,-0.071836,0.885390,-0.299687,0.168544,0.789741,0.809167,0.311725
4,1.492282,1.374795,0.686666,0.507509,-0.448246,0.789741,0.559051,0.311725
...,...,...,...,...,...,...,...,...
1454,2.438219,-1.518467,-0.041991,-1.166169,-0.806136,-1.026041,0.058817,-2.365440
1455,2.438219,-1.518467,-0.041991,-1.166169,-0.806136,-1.026041,-0.358044,-1.026858
1456,-0.872563,-0.795151,-0.373198,0.379817,-0.554851,-1.026041,-0.774905,0.311725
1457,0.664586,-0.795151,0.686666,-0.331610,-1.038384,-1.026041,0.058817,-2.365440


In [26]:
preds = model_selector.predict(X_test_prepared)
preds

array([120636.69157402, 150500.27667547, 171225.36889781, ...,
       134143.38938942, 114609.15947291, 235894.99537106])

In [27]:
preds_series = pd.Series(preds, name='SalePrice')
output =  pd.concat([test['Id'], preds_series], axis=1)
output.head()

,Id,SalePrice
0,1461,120636.691574
1,1462,150500.276675
2,1463,171225.368898
3,1464,180802.840228
4,1465,205052.413381


In [28]:
output.to_csv('housing_submission_pipeline.csv', index=False)

In [29]:
(0.15717-0.18558)/0.18558

-0.15308761720012928

Kaggle provided the score as 0.15717. Note that the previous model in my previous [Kaggle](https://www.kaggle.com/code/somebodynamedalexis/introduction-to-ml-prediction)/[Github](https://nbviewer.org/github/Zihac/Introduction-to-ML-Models/blob/main/ML_Prediction/house-price-prediction-begining.ipynb) code yielded a performance around 0.18558. This suggests that dropping the worst-performing base model increased the performance by approximately 15%.

# Grid Search

- In this section, I aim to improve the performance of the model by fine-tuning the parameters of the base models through grid search.

In [30]:
np.arange(2, 8, 2)

array([2, 4, 6])

In [31]:
param_grid = [
    {'rf__n_estimators': [3,10,30], 'rf__max_features':[2,4,6,8],'dt__max_depth':[6,8,10],'knn__n_neighbors': np.arange(2, 8, 1) }
]

In [32]:
grid_search = GridSearchCV(model_selector.get_stacking(), param_grid, cv=3, scoring='neg_mean_squared_error', n_jobs=-1)

In [34]:
# Fit the GridSearchCV
grid_search.fit(X_train_prepared, y_train)

GridSearchCV(cv=3,
             estimator=StackingRegressor(cv=5,
                                         estimators=[('rf',
                                                      RandomForestRegressor()),
                                                     ('knn',
                                                      KNeighborsRegressor()),
                                                     ('dt',
                                                      DecisionTreeRegressor())],
                                         final_estimator=LinearRegression()),
             n_jobs=-1,
             param_grid=[{'dt__max_depth': [6, 8, 10],
                          'knn__n_neighbors': array([2, 3, 4, 5, 6, 7]),
                          'rf__max_features': [2, 4, 6, 8],
                          'rf__n_estimators': [3, 10, 30]}],
             scoring='neg_mean_squared_error')

In [35]:
# Best parameters and model
best_params = grid_search.best_params_
best_params

{'dt__max_depth': 6,
 'knn__n_neighbors': 3,
 'rf__max_features': 2,
 'rf__n_estimators': 10}

In [36]:
best_model = grid_search.best_estimator_
best_model

StackingRegressor(cv=5,
                  estimators=[('rf',
                               RandomForestRegressor(max_features=2,
                                                     n_estimators=10)),
                              ('knn', KNeighborsRegressor(n_neighbors=3)),
                              ('dt', DecisionTreeRegressor(max_depth=6))],
                  final_estimator=LinearRegression())

In [37]:
preds = best_model.predict(X_test_prepared)
preds

array([119738.57949424, 149909.36818133, 172734.23308477, ...,
       140934.62830762, 115149.11718153, 231725.313826  ])

In [38]:
preds_series = pd.Series(preds, name='SalePrice')
output =  pd.concat([test['Id'], preds_series], axis=1)
output.head()

,Id,SalePrice
0,1461,119738.579494
1,1462,149909.368181
2,1463,172734.233085
3,1464,180051.374860
4,1465,195898.501199


In [39]:
output.to_csv('housing_submission_pipeline_Gridsearch.csv', index=False)

In [40]:
import joblib

# Save the stacking model
joblib.dump(best_model, 'stacking_model_GridSearch.pkl')

['stacking_model_GridSearch.pkl']

In [41]:
(0.15691-0.15717)/0.15717

-0.0016542597187759129

The new score is 0.15691, representing a 0.1% improvement from the stacking model without grid search for the best parameters.

# Reference
- If you have any questions about the reasons behind the steps, please check out my previous notebook on [Kaggle](https://www.kaggle.com/code/somebodynamedalexis/introduction-to-ml-prediction)/[Github](https://nbviewer.org/github/Zihac/Introduction-to-ML-Models/blob/main/ML_Prediction/house-price-prediction-begining.ipynb) as well as the reference
- For more information on model pipelines, you can also refer to the book Hands-On Machine Learning with Scikit-Learn, Keras, and TensorFlow by Aurélien Géron (2022), published by O'Reilly Media, Inc.